In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re
import gensim.models.word2vec as w2v
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import seaborn

In [2]:
df = pd.read_csv("preprocessed_news.csv")
df = df.iloc[:,[1,2]]
df.head()

,Article,Category
0,Paul Tudor Jones: The $19 trillion private sec...,Politics
1,AT&T CEO takes a swipe at new rival Netflix an...,Investing
2,Elon Musk's Tesla files for a trademark of 'Te...,Technology
3,"Cramer's game plan: In a controversial week, l...",Markets
4,Morgan Stanley is serious about investing in o...,Investing


In [3]:
df.describe()

,Article,Category
count,2686,2686
unique,2686,5
top,"Pepsi, Coke said to become a force behind init...",Investing
freq,1,579


In [4]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

  

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhishek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
corpus = []
for i in range(0,2686):
    review = re.sub('[^a-zA-Z]', ' ', df['Article'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english')) and len(review) > 2]
#     review = ' '.join(review)
    corpus.append(review)
print(corpus)

[['paul', 'tudor', 'jone', 'trillion', 'privat', 'sector', 'lead', 'social', 'chang'], ['ceo', 'take', 'swipe', 'new', 'rival', 'netflix', 'call', 'walmart', 'video', 'stream'], ['elon', 'musk', 'tesla', 'file', 'trademark', 'teslaquila'], ['cramer', 'game', 'plan', 'controversi', 'week', 'leav', 'room', 'strategi'], ['morgan', 'stanley', 'seriou', 'invest', 'outer', 'space', 'first', 'confer'], ['fiat', 'chrysler', 'plan', 'open', 'factori', 'detroit', 'build', 'new', 'three', 'row', 'jeep', 'grand', 'cheroke'], ['siri', 'creator', 'say', 'appl', 'drop', 'ball', 'third', 'parti', 'voic', 'partner', 'futur'], ['remington', 'rifl', 'settlement', 'includ', 'free', 'trigger', 'replac', 'offici'], ['stock', 'fall', 'tesla', 'cut', 'price', 'china', 'say', 'pay', 'us', 'custom', 'miss', 'tax', 'credit'], ['nielsen', 'get', 'seriou', 'go', 'privat', 'talk', 'potenti', 'buyer', 'januari'], ['croc', 'close', 'last', 'manufactur', 'plant', 'say', 'go', 'anywher'], ['cramer', 'remix', 'wast', 't

In [6]:
token_count = sum([len(sentence) for sentence in corpus])
print("The book corpus contains {0:,} tokens".format(token_count))


The book corpus contains 23,932 tokens


In [7]:
import multiprocessing
#ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [8]:
modelsu = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [9]:
modelsu.build_vocab(corpus)


In [10]:


print("Word2Vec vocabulary length:", len(modelsu.wv.vocab))

print(modelsu.wv.similarity(w1="bad",w2="good"))

Word2Vec vocabulary length: 1660
0.0824998


/home/abhishek/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [11]:
modelsu.train(corpus,total_examples=modelsu.corpus_count,epochs=modelsu.epochs)

(92222, 119660)

In [12]:
modelsu.most_similar(positive=['market'])

/home/abhishek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/abhishek/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lower', 0.9997742176055908),
 ('amid', 0.9997517466545105),
 ('higher', 0.9997368454933167),
 ('low', 0.9996961355209351),
 ('global', 0.9996837377548218),
 ('fear', 0.9996771216392517),
 ('stock', 0.9996728897094727),
 ('brexit', 0.9996662139892578),
 ('bank', 0.9996585845947266),
 ('itali', 0.9996574521064758)]

In [13]:
import os
if not os.path.exists("trained"):
    os.makedirs("trained")

In [14]:
modelsu.save(os.path.join("trained", "vec.w2v"))

In [15]:
thrones2vec = w2v.Word2Vec.load(os.path.join("trained", "vec.w2v"))